## importing the libraries

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalMaxPooling1D,Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

## importing the dataset

In [4]:
content = open('dataset_mentalDesorder.json')
data1 = json.load(content)

## getting all the data to lists

In [5]:
tags = []
inputs = []
responses = {}
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])

## converting to dataframe

In [6]:
data = pd.DataFrame({"input":inputs,"tag":tags})

## printing the data

In [7]:
data

,input,tag
0,what is Generalized Anxiety Disorder?,Generalized_Anxiety_Disorder
1,Generalized Anxiety Disorder,Generalized_Anxiety_Disorder
2,what is GAD?,Generalized_Anxiety_Disorder
3,GAD?,Generalized_Anxiety_Disorder
4,what is Panic Disorder?,Panic_disorder
...,...,...
67,Narcolepsy,Narcolepsy
68,what is Sleep Apnea?,Sleep_Apnea
69,Sleep Apnea,Sleep_Apnea
70,Bye,goodbye


## removing punctuations

In [8]:
import string
data['input'] = data['input'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['input'] = data['input'].apply(lambda wrd: ''.join(wrd))
data

,input,tag
0,what is generalized anxiety disorder,Generalized_Anxiety_Disorder
1,generalized anxiety disorder,Generalized_Anxiety_Disorder
2,what is gad,Generalized_Anxiety_Disorder
3,gad,Generalized_Anxiety_Disorder
4,what is panic disorder,Panic_disorder
...,...,...
67,narcolepsy,Narcolepsy
68,what is sleep apnea,Sleep_Apnea
69,sleep apnea,Sleep_Apnea
70,bye,goodbye


## tokenize the data

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['input'])
train = tokenizer.texts_to_sequences(data['input'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tag'])

In [10]:
input_shape = x_train.shape[1]
print(input_shape)

6


## define vocabulary

In [11]:
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = le.classes_.shape[0]
print("output length: ",output_length)

number of unique words :  57
output length:  29


In [12]:
def get_model():
    #creating the model
    i = Input(shape=(input_shape,))
    x = Embedding(vocabulary+1,10)(i)
    x = LSTM(10,return_sequences=True)(x)
    x = Flatten()(x)
    x = Dense(output_length,activation="softmax")(x)
    model = Model(i,x)
    return model

In [13]:
model = get_model()

## compiling the model

In [14]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

## training the model

In [31]:
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
3/3 [==============================] - 4s 7ms/step - loss: 3.3682 - accuracy: 0.0000e+00
Epoch 2/200
3/3 [==============================] - 0s 6ms/step - loss: 3.3660 - accuracy: 0.0278
Epoch 3/200
3/3 [==============================] - 0s 6ms/step - loss: 3.3645 - accuracy: 0.0556
Epoch 4/200
3/3 [==============================] - 0s 6ms/step - loss: 3.3632 - accuracy: 0.0556
Epoch 5/200
3/3 [==============================] - 0s 7ms/step - loss: 3.3618 - accuracy: 0.0972
Epoch 6/200
3/3 [==============================] - 0s 9ms/step - loss: 3.3604 - accuracy: 0.1389
Epoch 7/200
3/3 [==============================] - 0s 7ms/step - loss: 3.3589 - accuracy: 0.1250
Epoch 8/200
3/3 [==============================] - 0s 8ms/step - loss: 3.3574 - accuracy: 0.0972
Epoch 9/200
3/3 [==============================] - 0s 8ms/step - loss: 3.3558 - accuracy: 0.1111
Epoch 10/200
3/3 [==============================] - 0s 9ms/step - loss: 3.3542 - accuracy: 0.1111
Epoch 11/200
3/3 [=======

Epoch 167/200
3/3 [==============================] - 0s 5ms/step - loss: 1.0154 - accuracy: 0.8889
Epoch 168/200
3/3 [==============================] - 0s 6ms/step - loss: 1.0039 - accuracy: 0.8889
Epoch 169/200
3/3 [==============================] - 0s 6ms/step - loss: 0.9915 - accuracy: 0.9028
Epoch 170/200
3/3 [==============================] - 0s 5ms/step - loss: 0.9808 - accuracy: 0.9028
Epoch 171/200
3/3 [==============================] - 0s 5ms/step - loss: 0.9719 - accuracy: 0.9028
Epoch 172/200
3/3 [==============================] - 0s 6ms/step - loss: 0.9618 - accuracy: 0.9028
Epoch 173/200
3/3 [==============================] - 0s 7ms/step - loss: 0.9516 - accuracy: 0.9167
Epoch 174/200
3/3 [==============================] - 0s 5ms/step - loss: 0.9422 - accuracy: 0.9167
Epoch 175/200
3/3 [==============================] - 0s 5ms/step - loss: 0.9305 - accuracy: 0.9167
Epoch 176/200
3/3 [==============================] - 0s 5ms/step - loss: 0.9199 - accuracy: 0.9167
Epoch 177/

## plotting model accuracy

In [16]:
plt.plot(train.history['accuracy'],label='training set accuracy')
plt.plot(train.history['loss'],label='training set loss')
plt.legend()

AttributeError: 'list' object has no attribute 'history'

## Saving model

In [17]:
model.save("ChatBot001.h5")

# Testing Model

In [ ]:
# Testing Model
# chatting
import random
from tensorflow import keras


#Loading model
reconstructed_model = keras.models.load_model("ChatBot001.h5")


while True:
    texts_p = []
    prediction_input = input('You : ')
    
    #removing punctuation and converting to lowercase
    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)
    
    #tokenizing and padding
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input],input_shape)
    
    #getting output from model
    output = reconstructed_model.predict(prediction_input)
    output = output.argmax()
    
    #finding the right tag and predicting
    response_tag = le.inverse_transform([output])[0]
    print("Chat Bot :",random.choice(responses[response_tag]))
    if response_tag == "goodbye":
        break

You : what is Panic Disorder?
1/1 [==============================] - 2s 2s/step
Chat Bot : Persistent Depressive Disorder, also known as dysthymia, is a chronic form of depression characterized by a persistently depressed mood that lasts for at least two years. Individuals with this disorder may experience a range of depressive symptoms, such as low self-esteem, sleep disturbances, poor appetite, and low energy. Treatment typically involves therapy and, in some cases, medication.
You : what is Social Anxiety Disorder?
1/1 [==============================] - 0s 115ms/step
Chat Bot : Schizoaffective Disorder is a mental disorder characterized by a combination of symptoms of schizophrenia, such as hallucinations and delusions, along with symptoms of a mood disorder, such as mania or depression. Individuals with this disorder experience both psychotic and mood-related symptoms, which can vary in severity. Treatment often involves a combination of medications and therapy.
You : what is Speci